In [ ]:
%matplotlib widget

In [ ]:
%%javascript
// Disable scrolling on plots
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import json
import re
from pathlib import Path

import ipywidgets as widgets
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.display import HTML, display
from IPython.display import JSON, display
from ipywidgets import fixed, interact, interact_manual, interactive
from matplotlib.ticker import (AutoMinorLocator, FormatStrFormatter,
                               MultipleLocator)

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.precision", 2)
pd.options.display.float_format = "{0:,.6f}".format

In [ ]:
# Make cells wider
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
RESULTS_DIR = Path("./results")
METHODS = ("qsort_c", "vanilla_quicksort", "insertion_sort", "std")
THRESHOLD_METHODS = ("qsort_c",)
DEFAULT_METHOD = "qsort_c"
INPUT_TYPES = ("ascending", "descending", "random", "single_num", "N/A")
FIG_SIZE = (10, 4)


def clean_path(path: Path):
    for i in INPUT_TYPES:
        if i in path:
            return re.sub(f"^.*?/{i}", str(i), path)
    return path


def load(in_dir=None):
    global df, avg_df
    if in_dir is None:
        # Find the latest results directory
        dirs = list(RESULTS_DIR.iterdir())
        dirs.sort()
        try:
            in_dir = dirs[-1]
        except IndexError as e:
            raise IndexError("No results found") from e

    csvs = list(in_dir.glob("*.csv"))
    if len(csvs) < 1:
        raise FileNotFoundError("CSV missing")
    in_file = csvs[0]

    # If possible, load the system details
    try:
        info_path = Path(in_dir, "job_details.json")
        if info_path.exists() and info_path.is_file():
            with open(info_path, "r") as json_file:
                info = json.load(json_file)
            for i in info:
                print(f"{i}: {str(info[i]).rstrip()}")
    # TODO: Make this the proper exception
    except Exception:
        print("Couldn't load system details")

    # If possible, load the partition
    partition_path = Path(in_dir, "partition")
    if partition_path.is_file():
        print(f"Partition: {partition_path.read_text()}")
    else:
        print("Partition: none")
    
    # Read the actual data
    df = pd.read_csv(in_file)

    # Convert microseconds to milliseconds and rename column
    df["Elapsed Time (microseconds)"] = df["Elapsed Time (microseconds)"] / 1000
    df.rename(
        columns={"Elapsed Time (microseconds)": "Runtime (milliseconds)"}, inplace=True
    )
    # Add seconds column
    df["Runtime (seconds)"] = df["Runtime (milliseconds)"] / 1000

    # Reorder columns
    df = df[
        [
            "Input",
            "Description",
            "Method",
            "Size",
            "Threshold",
            "Runtime (milliseconds)",
            "Runtime (seconds)",
        ]
    ]

    # Cleanup the input column
    df["Input"] = df["Input"].apply(clean_path)

    display(df[["Runtime (milliseconds)", "Runtime (seconds)"]].describe())

    # Average runtime for repeated runs.
    avg_df = (
        df.groupby(["Input", "Method", "Description", "Size", "Threshold"])
        .mean()
        .reset_index()
    )

    print(f"Loaded: {in_file}")

    # Get min and max threshold for interactions
    default_df = df[df["Method"] == DEFAULT_METHOD]
    thresholds = sorted(set(df["Threshold"]))
    try:
        thresholds.remove(0)
    except ValueError:
        pass

    # TODO: rework this to support any number of methods (widgets.SelectMultiple)
    interact_manual(
        plot_vs_methods_helper,
        method=widgets.Dropdown(
            options=METHODS, value=METHODS[0], description="Method 1:"
        ),
        method2=widgets.Dropdown(
            options=METHODS, value=METHODS[-1], description="Method 2:"
        ),
        unit=widgets.Dropdown(options=units, description="Unit"),
        thresh=widgets.SelectionSlider(options=thresholds, description="Threshold"),
        table=widgets.Checkbox(description="Table"),
        continuous_update=False,
    )

    default_df = df[df["Method"] == "qsort_c"]
    sizes = sorted(set(default_df["Size"]))

    interact_manual(
        plot_threshold_impact,
        name="Threshold Impact",
        unit=widgets.Dropdown(options=units, description="Unit"),
        size=widgets.SelectionSlider(options=sizes),
        error_bars=widgets.Checkbox(description="Error Bars"),
        table=widgets.Checkbox(description="Table"),
        descrip=widgets.Checkbox(description="Data Description"),
        continuous_update=False,
    )

In [ ]:
def plot_vs_methods(
    *methods,
    thresh=4,
    unit="milliseconds",
    table=False,
):
    unit = unit.lower()

    thresholds: list[int] = []
    method_strs: list[str] = []
    types = sorted(set(df["Description"]))

    for m in methods:
        if m in THRESHOLD_METHODS:
            thresholds.append(thresh)
            method_strs.append(f"{m} (threshold: {thresh:})")
        else:
            thresholds.append(0)
            method_strs.append(m)

    for input_t in types:
        fig, ax = plt.subplots(figsize=FIG_SIZE)
        for k, m in enumerate(methods):
            my_df = avg_df[
                (avg_df["Method"] == m)
                & (avg_df["Description"] == input_t)
                & (avg_df["Threshold"] == thresholds[k])
            ]

            title = " vs. ".join(method_strs)
            title += f"\n{input_t}"

            my_df = my_df.sort_values(["Description", "Method", "Threshold", "Size"])
            
            
            my_df.plot(
                ax=ax,
                x="Size",
                y=f"Runtime ({unit})",
                marker=".",
                markersize=10,
                title=title,
                grid=True,
            )
            
            if table:
                display(my_df)
                
        ax.legend(methods)


def plot_vs_methods_helper(
    method=METHODS[0], method2=METHODS[-1], thresh=4, unit="milliseconds", table=False
):
    plot_vs_methods(*[method, method2], thresh=thresh, unit=unit, table=table)

In [ ]:
def plot_threshold_impact(
    size=50_000,
    unit="milliseconds",
    error_bars=False,
    table=False,
    descrip=False,
):
    unit = unit.lower()
    dfs = []

    for i in INPUT_TYPES:
        m = df[df["Method"] == "qsort_c"]
        d = m[m["Description"] == i]
        frame = d[d["Size"] == size]

        dfs.append(frame)

    for frame in dfs:
        # Skip if empty frame
        if not len(frame):
            continue

        # Sort for pretty graphs
        frame = frame.sort_values(["Description", "Method", "Threshold", "Size"])

        # Error bar calc
        group = frame.groupby(["Input", "Method", "Description", "Size", "Threshold"])
        means = group.mean().reset_index()
        errors = 2 * group.std().reset_index()

        # Select the first row to get some more info
        row_0 = frame.iloc[0]

        # Assemble the title
        title = f"Qsort_c Threshold Value vs. Runtime at {size:,} Inputs\n{row_0['Description'].capitalize()}"

        # Plot
        if error_bars:
            ax = means.plot(
                x="Threshold",
                y=f"Runtime ({unit})",
                yerr=errors,
                capsize=4,
                ecolor="black",
                marker=".",
                markersize=10,
                title=title,
                grid=True,
                figsize=FIG_SIZE,
            )
        else:
            ax = means.plot(
                x="Threshold",
                y=f"Runtime ({unit})",
                marker=".",
                markersize=10,
                title=title,
                grid=True,
                figsize=FIG_SIZE,
            )

        # Set axis titles
        ax.set_xlabel("Threshold")
        ax.set_ylabel(f"Runtime ({unit})")

        # Force int x axis
        ax.get_xaxis().set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))

        # Table of data
        if table:
            display(frame)

        if descrip:
            display(frame[[f"Runtime ({unit})"]].describe())

In [ ]:
units = ["seconds", "milliseconds"]

results_dir = Path("./results")
dirs = list(sorted([x for x in results_dir.iterdir()]))

# Give the option to load any data set
interact(
    load,
    in_dir=widgets.Dropdown(
        options=dirs,
        value=dirs[-1],
        description="Results Dir:",
    ),
)